In [6]:
import rasterio
import numpy as np
import folium
import cv2  # OpenCV for image processing
year=str(2015)
# File path
raster_path = f"/Users/jorgeandresjolahernandez/Desktop/Proyecto_Arauca/Automation_App/Image_Extraction_Arauca/static/rasters/Saravena/{year}.tif"

# Open the raster and read the bands
with rasterio.open(raster_path) as src:
    band1 = src.read(3)  # Red
    band2 = src.read(2)  # Green
    band3 = src.read(1)  # Blue
    profile = src.profile
    bounds = src.bounds

# Function to stretch values using percentiles (for better contrast)
def percentile_stretch(band, lower_percentile=2, upper_percentile=98):
    # Calculate the lower and upper percentiles
    pmin = np.percentile(band, lower_percentile)
    pmax = np.percentile(band, upper_percentile)
    
    # Stretch values to the range 0-255 based on percentiles
    stretched_band = ((band - pmin) / (pmax - pmin) * 255).clip(0, 255)
    return np.uint8(stretched_band)

# Apply percentile-based contrast stretching to each band
band1_stretched = percentile_stretch(band1)
band2_stretched = percentile_stretch(band2)
band3_stretched = percentile_stretch(band3)

# Stack the stretched bands to form a color image (RGB)
rgb_image = np.stack([band1_stretched, band2_stretched, band3_stretched], axis=-1)

# Apply histogram equalization to improve contrast
rgb_image_eq = cv2.equalizeHist(rgb_image[:, :, 0])  # Red channel
band2_eq = cv2.equalizeHist(rgb_image[:, :, 1])  # Green channel
band3_eq = cv2.equalizeHist(rgb_image[:, :, 2])  # Blue channel

# Stack the equalized bands to form the image
rgb_image_eq = np.stack([rgb_image_eq, band2_eq, band3_eq], axis=-1)

# Create an alpha channel where values are transparent only if they are 0
alpha_channel = np.where((band1 == 0) & (band2 == 0) & (band3 == 0), 0, 255).astype(np.uint8)

# Add transparency to the image using the alpha channel
rgba_image = cv2.merge([rgb_image_eq[:, :, 0], rgb_image_eq[:, :, 1], rgb_image_eq[:, :, 2], alpha_channel])

# Create a temporary file for the RGBA image (for folium overlay)
output_path = "overlay_image.png"
cv2.imwrite(output_path, rgba_image)

# Create a folium map centered around the raster bounds
map_center = [(bounds[1] + bounds[3]) / 2, (bounds[0] + bounds[2]) / 2]
m = folium.Map(location=map_center, zoom_start=10)

# Add the image as an overlay to the folium map with adjusted opacity
folium.raster_layers.ImageOverlay(
    image=rgba_image,
    bounds=[[bounds[1], bounds[0]], [bounds[3], bounds[2]]],
    opacity=0.9  # Adjusted opacity to make the image more visible
).add_to(m)

# Save the folium map as an HTML file
map_html_path = f"{year}.html"
m.save(map_html_path)

# Print the path to the HTML file
print(f"Map saved as HTML at: {map_html_path}")


Map saved as HTML at: 2015.html


/var/folders/j9/v4x3hnqx5mx5xp1r2572v97c0000gn/T/ipykernel_2763/3435597596.py:24: RuntimeWarning: divide by zero encountered in divide
  stretched_band = ((band - pmin) / (pmax - pmin) * 255).clip(0, 255)
/var/folders/j9/v4x3hnqx5mx5xp1r2572v97c0000gn/T/ipykernel_2763/3435597596.py:24: RuntimeWarning: invalid value encountered in divide
  stretched_band = ((band - pmin) / (pmax - pmin) * 255).clip(0, 255)
/var/folders/j9/v4x3hnqx5mx5xp1r2572v97c0000gn/T/ipykernel_2763/3435597596.py:25: RuntimeWarning: invalid value encountered in cast
  return np.uint8(stretched_band)
